In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('/Users/taewon/Documents/취준/대구은행_프로젝트/중소기업은행_금융용어해설(퇴직연금).csv', encoding='euc-kr')

In [28]:
df.drop(columns=['Unnamed: 2'], inplace=True)

In [29]:
df.to_csv('금융용어_해설.csv', index=False, encoding='utf-8')

In [30]:
df2 = pd.read_csv('/Users/taewon/Documents/취준/대구은행_프로젝트/금융용어_해설.csv', encoding='utf-8')

In [32]:
df2

,용 어,설 명
0,가입기간,"퇴직연금 설정 이후 당해 사업장에서 근로를 제공한 기간, 당해 퇴직연금의 설정 전에..."
1,가입연령,퇴직연금에 가입 할 수 있는 최소연령
2,가입자,퇴직연금에 가입하여 연금규약에 따라 퇴직급여 수급권을 취득한 근로자 또는 개인퇴직계...
3,가입자 자격상실,"퇴직연금 가입자가 정년, 중도사망, 중도퇴직 등의 사유로 가입자로서의 자격을 상실하..."
4,가입자명부,퇴직연금에 가입한 사업장에 속해있는 근로자의 퇴직금정보 등을 포함한 기본사항을 적은 명부
...,...,...
191,CD,정기예금을 양도가 가능하게 하여 문서로 증서화 함으로써 유동성을 부여한 무기명 정기...
192,CMA,예탁금을 어음이나 채권에 투자하여 그 수익을 고객에게 돌려주는 실적배당 금융상품
193,EET 세제체계(EET system),부담금 납입단계와 적립금 운용단계에서 비과세 후 연금급여 수령단계에서 연금소득으로 ...
194,이율보증형(GIC),생명보험 회사가 퇴직연금기금 등에서 자금운용을 위탁 받아 미리 약정한 원리금을 기금...


In [34]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/polyglot-ko-12.8b-safetensors"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0})

import pandas as pd

from datasets import load_dataset

data = load_dataset("json",
                    data_files={"train":"/content/drive/MyDrive/금융용어_해설.json"})

[row for row in data['train']]

data = data.map(
    lambda x: {
        'text': f"""User: {x['용 어']}

Assistant: {x['설 명']}<|endoftext|>""" }
)

[row['text'] for row in data['train']]

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

data["train"][0]["text"]

tokenized_text = tokenizer(data["train"][0]["text"])
tokenized_text

token_text = [tokenizer.decode(id) for id in tokenized_text["input_ids"]]
", ".join(token_text)

from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=150,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

trainable_params = 0
all_param = 0
for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()

print(f"""trainable params: {trainable_params}
all params: {all_param}
trainable%: {100 * trainable_params / all_param}""")

import transformers

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        max_steps=150, # 100 step 학습시, V100 기준 약 2분 소요
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        learning_rate=1e-4,
        fp16=True,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        logging_steps=25,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

model.eval()
model.config.use_cache = True

from transformers.generation import GenerationConfig

gen_cfg = GenerationConfig.from_model_config(model.config)
gen_cfg.pad_token_id = tokenizer.eos_token_id

def gen(x):
    gen_cfg.temperature = 0.1
    gen_cfg.max_new_tokens = 128

    gened = model.generate(
        **tokenizer(
            f"User: {x}\n\nAssistant:",
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        do_sample=True,
        generation_config=gen_cfg
    )

    return tokenizer.decode(gened[0])

gen('사모펀드')



NameError: name '_C' is not defined